In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.filters as flt
from skimage.metrics import structural_similarity as SSIM
%matplotlib inline
# since we can't use imports
import numpy as np
import scipy.ndimage.filters as flt
import warnings
import cv2
from glob import glob 

import pylab as pl
from time import sleep
import tensorflow as tf
from tqdm import tqdm
from collections import Counter
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib as mpl


import sys
sys.path.append('../../scripts')
import open_frame
from pm_algorithm import anisodiff
from data_augmentation import get_generators
from architectures import get_model


CROP = 256
image_size = (CROP,CROP)
typ = "gaussian2"
open_frame = getattr(open_frame,typ)


import logging
tf.get_logger().setLevel(logging.ERROR)

In [2]:
def loss(y_true,y_pred):
    return 1/(tf.image.psnr(y_true,y_pred,1)+1e-16) + 1/(tf.math.tan(np.pi*tf.image.ssim(y_true,y_pred,1)/2)+1e-16)

In [21]:
typ = "gaussian"
gen_batch_train,gen_batch_val = get_generators(typ,var1_d=1,var1_u=7,CROP1=CROP,BATCH_SIZE=50)

order = 1
for num_classes in [10,15,20,50]:
    print('num_classes: ',num_classes)
    model = get_model("splines",it_lim=1,image_size=image_size,num_classes = num_classes,order=order)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
            loss=loss,)
    #model.load_weights(f"./checkpoints/splines_{typ}_{num_classes}")
    
    callbacks = [tf.keras.callbacks.ModelCheckpoint(
    filepath= f"./checkpoints/splines_{typ}_{num_classes}",
    save_weights_only=True,
    verbose = True,
    save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, verbose=1)
    ]
    
    history = model.fit(
        gen_batch_train,
        epochs=50,
        steps_per_epoch=10,
        validation_data=gen_batch_val,
        validation_steps=5,
        shuffle=False,
        use_multiprocessing=True,
        callbacks=callbacks,
        workers=1
    )

num_classes:  10
Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 2.1642
Epoch 00001: val_loss improved from inf to 2.42744, saving model to ./checkpoints/splines_gaussian_10
10/10 [==============================] - 15s 1s/step - loss: 2.1642 - val_loss: 2.4274 - lr: 0.0010
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 1.7917
Epoch 00002: val_loss improved from 2.42744 to 2.39756, saving model to ./checkpoints/splines_gaussian_10
10/10 [==============================] - 12s 1s/step - loss: 1.7917 - val_loss: 2.3976 - lr: 0.0010
Epoch 3/50
 4/10 [===========>..................] - ETA: 5s - loss: 1.6834

KeyboardInterrupt: 

In [16]:
xs, ys = next(gen_batch_val.as_numpy_iterator())

In [17]:
get_lambdas = tf.keras.models.Model(model.input,model.get_layer('coeff_x_0').output)
spline = np.squeeze(get_lambdas(xs))

In [18]:
spline

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan